## 1. Latent Dirichlet Allocation

As a final, more advanced, example of variational inference, we can consider Latent Dirichlet Allocation (LDA).  LDA is a Bayesian topic model that uses variational inference in the original paper to perform posterior inference for the parameters.  It is a very widely used model and many extensions to more complicated settings, such as dynamic topic models, also exist.

The data generating process described for the original LDA model (http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf) is as follows:

1. Choose the number of topics N, $N\sim\mbox{Poisson}(\epsilon)$.

2. Choose topic level parameters $\theta$, $\theta\sim\mbox{Dirichlet}(\alpha)$.

3. For each of the $n$ words $w_n$:
    
    a. Choose a topic $z_n\sim\mbox{Multinomial}(\theta)$
    
    b. Choose a word $w_n$ from a multinomial probability conditioned on the topic $z_n$, $p(w_n | z_n, \beta)$

The joint distribution of the model is then

$$p(\theta, z, w, |\alpha, \beta) = p(\theta | \alpha)\prod_{n=1}^N p(z_n | \theta)p(w_n |z_n, \beta).$$

The parameters of interest exist at several levels in the document corpus.  At the lowest level, the parameters $z_{dn}$ and $w_{dn}$ are at the word-level and are sampled once for each word $n$ in each document $d$.  At the document level are the parameters $\theta_d$, which are sampled once for each document $d$. Finally, at the overall corpus level are the parameters $\alpha$ and $\beta$, which are sampled once to generate the corpus of documents.

The variational distribution for inference is

$$q(\theta, z |\gamma, \phi) = q(\theta|\gamma)\prod_{n=1}^N q(z_n |\phi_n).$$

We can also obtain empirical Bayes estimates for $\alpha$ and $\beta$ by maximizing the variational lower bound for these parameters.

Full derivations for all steps of the inference can be found in the original paper (http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf).  A summary of the inference steps is presented here.

Let $k$ be the dimension of the topic parameter $z_n$, so $k$ is the number of topics and is specified ahead of the inference.  $V$ is the total number of words in the vocabulary across the corpus. There are assumed to be $M$ documents in the corpus and $N$ words in each document, where $N$ can be sampled from a Poisson distribution as described in the data generating process above.

**Variational E-Step:** The goal for the E-step is to find the optimizing values of the variational parameters, $\gamma_d^*$ and $\phi_d^*$ for each document $d$:

1. Initialize $\phi_{ni} = \dfrac{1}{k}$ for all $i$ and $n$
2. Initialize $\gamma_i = \alpha_i + \dfrac{N}{k}$ for all $i$
3. Until convergence, repeat for $n = 1, \ldots, N$:

    a. For $i = 1, \ldots, k$: $$\phi_{ni} = \beta_{iw_n}\mbox{exp}(\psi(\gamma_i))$$
        
    b. Normalize $\phi_n$ to sum to 1
    
    c. $$\gamma = \alpha + \sum_{n=1}^N\phi_n$$
    
    
    

**Variational M-Step:** Goal: update the parameters $\alpha$ and $\beta$.

\begin{aligned}
\beta_{ij} &\propto \sum_{d=1}^M\sum_{n=1}^{N_d}\phi_{ni}w_{dn}^j \\
\alpha_{new} = \alpha_{old} - H(\alpha_{old})^{-1}g(\alpha_{old}) \\
(H^{-1}g)_i &= \dfrac{g_i - c}{h_i} \\
c &= \dfrac{\sum_{j=1}^k g_j/h_j}{z^{-1} + \sum h_j^{-1}} \\
g_j &= M\left(\psi\left(\sum_{i=1}^k\alpha_i\right) - \psi(\alpha_j)\right) + \sum_{d=1}^M\left(\psi(\gamma_{di}) - \psi\left(\sum_{j=1}^k \gamma_{dj}\right)\right) \\
h_j &= M\psi'(\alpha_j) \\
z &= \psi'\left(\sum_{j=1}^k \alpha_j\right)
\end{aligned}


## 2. Implementation

We will implement LDA for some toy data. Below are four sentences, each of which is treated as a document. There are then four documents in this toy corpus.  

Note: The code is implemented for clarity and not for efficiency.

In [1]:
import collections
s1 = "The quick brown fox"
s2 = "The brown fox jumps over the fence"
s3 = "The grey elephant sleeps."
s4 = "The dog, peacock, lion, tiger and elephant are friends."

docs = collections.OrderedDict()
docs["s1"] = s1
docs["s2"] = s2
docs["s3"] = s3
docs["s4"] = s4

In [2]:
import string
import stop_words
import numpy as np
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from scipy import special


#Function to make document, word matricies for LDA - performs stemming and tokenization#
def make_word_matrix(corpus, needToSplit):
    
    #define dictionary to store "cleaned" words
    cleanCorpus = {}
    #Define stop words
    stopWords = get_stop_words('english')
    
    
    #Initialize stemmer
    p_stemmer = PorterStemmer()

    
    #Define list to store corpus data#
    c = []
    #Define list to store order of words for each document#
    wordOrder = []
    #Define table to remove punctuation
    table = dict.fromkeys(map(ord, string.punctuation))

    M = len(corpus)
    
    #Check to make sure that dictionary isn't empty#
    if M ==0:
        print("Input dictionary is empty")
        return;
    
    removePunc = string.punctuation
    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        
        #if the documents in the corpus are contained in a single string
        if needToSplit == 1:
            #Remove punctuation 
            text = corpus[i].translate(table)
            #Splits string by blankspace and goes to lower case#
            words = text.lower().split()
        
        else:
            #Remove punctuation
            for j in range(0, len(removePunc)):
                while removePunc[j] in corpus[i]: 
                    corpus[i].remove(removePunc[j])    
            
            #convert everything to a lower case
            corpus[i] = list(map(lambda x:x.lower(),corpus[i]))
            words = corpus[i]

        #Remove stop words#
        text = [word for word in words if word not in stopWords]
        # stem tokens
        text = [p_stemmer.stem(i) for i in text]
        #Find total number of words in each document#
        N = len(text)
        cleanCorpus[i] = text
        #Find number of unique words in each document#
        Vwords = list(set(text))
        wordOrder.append(Vwords)

    #Find unique words in the corpus, this is the vocabulary#    
    wordOrder = list(set(x for l in wordOrder for x in l))
    wordOrder = sorted(wordOrder)
    #Find the number of unique words in the corpus vocabulary#
    V = len(wordOrder)
    
    #For each document in docs, caculate frequency of the words#
    for i in cleanCorpus:
        text = cleanCorpus[i]
        N = len(text)
        #Create matrix to store words for each document#
        wordsMat = np.zeros((N, V), dtype = int)
        count = 0
        for word in text:
            #Find which word in vocabulary current word in document corresponds to#
            v = wordOrder.index(word)
            #Add a 1 to that column in the wordsMat matrix#
            wordsMat[count, v] = 1
            count += 1
        c.append(wordsMat)

    return [c, wordOrder, M]

In [3]:
testMat = make_word_matrix(docs, 1)
testMat

[[array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]),
  array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
  array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]),
  array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
         [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])],
 ['brown',
  'dog',
  'eleph',
  'fenc',
  'fox',
  'friend',
  'grey',
  'jump',
  'lion',
  'peacock',
  'quick',
  'sleep',
  'tiger'],
 4]

In [4]:
## Perform LDA E-step for one document
## K is the number of topics
## N is the number of words in this document
## Alpha and beta are current parameter values
## doc is the current document
def Estep(K, N, V, alpha, beta, doc, tol):    
        
    #initialize phi and gamma
    oldPhi  = np.full(shape = (N,K), fill_value = 1/K)
    gamma = alpha + N/K
    converge = 0 
    
    count = 0
    
    while converge == 0:
        newPhi  = np.zeros(shape = (N,K))
        #Update phi
        for n in range(N):
            ind = np.where(doc[n,:] == 1)[0]
            newPhi[n,:] = [beta[i, ind]*np.exp(special.psi(gamma[i])) for i in range(K)]
        newPhi = newPhi/np.sum(newPhi, axis = 1)[:, np.newaxis] #normalizing the rows of new phi

        gamma = alpha + np.sum(newPhi, axis = 0)  #updating gamma

        #Check convergence criteria
        criteria = (1/(N*K))*np.sum((newPhi - oldPhi)**2)**0.5
        if criteria < tol:
            converge = 1
        else:
            oldPhi = newPhi
            count = count +1
            converge = 0
    return (newPhi, gamma)

In [5]:
#Update alpha using linear Newton-Rhapson Method#
## Gamma is a M x K array now
def alphaUpdate(K, M, alphaOld, gamma, tol):
    h = np.zeros(K)
    g = np.zeros(K)
    alphaNew = np.zeros(K)

    step = np.zeros(K)
    
    
    converge = 0
    while converge == 0:
        
        alpha_sum = np.sum(alphaOld)
        z = -special.polygamma(1, alpha_sum)
        h = M*special.polygamma(1, alphaOld)
        
        gam_sum = [special.psi(np.sum(gamma[d, :])) for d in range(M)]
        doc_sum = special.psi(np.sum(gamma, axis = 0)) - np.sum(gam_sum)
        g = np.array([M*(special.psi(alpha_sum)-special.psi(alphaOld[i])) + doc_sum[i] for i in range(K)])
        
        
        c = np.sum(g/h)/(1/z + np.sum(1/h))
        step = (g-c)/h
        
        # step = (g - c)/h

        if np.linalg.norm(step) < tol:
            converge = 1
        else:
            converge = 0
            alphaNew = alphaOld + step
            alphaOld = alphaNew

    return abs(alphaNew)

# V is the number of words in the vocabulary
# phi_list is a list of the phi matrices, one for each document
# Word list is a list of the word matrix for each document
def betaUpdate(K, M, V, phi_list, word_list):

    
    #Calculate beta#
    beta = np.zeros(shape = (K,V))
    Nd = [word_list[d].shape[0] for d in range(M)] # Number of words for document d

    for i in range(K):
        for j in range(V):
            wordSum = np.sum([phi_list[d][n,i]*word_list[d][n,j] for d in range(M) for n in range(Nd[d])])
            beta[i,j] = wordSum
    
    
    #Normalize the rows of beta#
    beta = beta/np.sum(beta, axis = 1)[:, np.newaxis]

    return beta

In [6]:
# K = number of topics 
# M = number of documents
# corpus matrix is output of make_word_matrix
# tol is the desired tolerance for convergence
def LDA(K, M, corpusMatrix, tol):

    M = corpusMatrix[2]
    output = []
    
    converge = 0
    #initialize alpha and beta for first iteration
    alphaOld = np.full(shape = K, fill_value = 50/K) + np.random.rand(K)
    V = corpusMatrix[0][0].shape[1]
    betaOld = np.random.rand(K, V)
    betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]
    
    iteration = 0
    while converge == 0:
        phi = []
        gamma = []
        
        ##E-step##
        #looping through the number of documents
        ## Perform LDA E-step for one document
        for d in range(M):
            N = corpusMatrix[0][d].shape[0]
            phiT, gammaT = Estep(K, N, V, alphaOld, betaOld, corpusMatrix[0][d], tol)
            phi.append(phiT)
            gamma.append(gammaT)
        print("E-step done")    
        ##M - step##
        #Update alpha and beta#     
        gamma = np.array(gamma)
        alphaNew = alphaUpdate(K, M, alphaOld, gamma, 1)
        betaNew = betaUpdate(K, M, V, phi, corpusMatrix[0])
        print("M-step done")
    
        if np.linalg.norm(alphaOld - alphaNew) < tol or np.linalg.norm(betaOld - betaNew) < tol:
            converge =1
        else: 
            converge =0
            iteration += 1
            alphaOld = alphaNew
            betaOld = betaNew
            print(iteration)
    output.append([phi, gamma, alphaNew, betaNew])
        
    return output

In [7]:
K = 3
M = testMat[2]
tol = 0.01
toy = LDA(K, M, testMat, tol)

E-step done
M-step done
1
E-step done
M-step done
2
E-step done
M-step done


In [8]:
import pandas as pd
#function to return the most probable words
#p is the number of words you want returned for each topic
def mostCommon(beta, wordList, p):
    k = beta.shape[0]
    topicWords = []
    betaDF = pd.DataFrame(beta)
    for i in range(0, k):
        document = betaDF.loc[i,:]
        ind = np.array(betaDF.iloc[0].sort_values(ascending = False).index[:p])
        mostCommon = pd.DataFrame(np.array(wordList)[ind])
        topicWords.append(mostCommon)
    return(topicWords)

In [9]:
toy_topics = mostCommon(toy[0][3], testMat[1], 3)

In [10]:
toy_topics[0]

,0
0,brown
1,fox
2,friend


## References

Blei, David M., Ng, Andrew Y. and Jordan, Michael I. *Latent Dirichlet Allocation*. Journal of Machine Learning Research 3 (2003) 993-1022. http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf